## Bert - POC

In [8]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

import os
import logging
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [9]:
class BertBaseMultilingualEmbeddingApi:
    
    def __init__(self, model_name="bert-base-multilingual-cased", cuda=True):
        self.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name).to(self.device)
        self.model.eval()
        
        self.tokens_tensor = None
        self.segments_tensor = None
        
        self.encoded_layers_ = None
        self.token_embeddings_ = None
        
    def _tokenize_text(self, text):
        marked_text = "[CLS] " + text + " [SEP]"
        tokenized_text = self.tokenizer.tokenize(marked_text)
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
        segments_ids = [1] * len(tokenized_text)
        
        self.tokens_tensor = torch.tensor([indexed_tokens]).to(self.device)
        self.segments_tensor = torch.tensor([segments_ids]).to(self.device)
    
    def _evaluate(self):
        with torch.no_grad():
            encoded_layers, _ = self.model(self.tokens_tensor, self.segments_tensor)
        self.encoded_layers_ = encoded_layers
    
    def _generate_token_embeddings(self, batch_i=0):
        """
        Convert the hidden state embeddings into single token vectors
        Holds the list of 12 layer embeddings for each token
        Will have the shape: [# tokens, # layers, # features]
        """
        token_embeddings = [] 
        # For each token in the sentence...
        for token_i in range(len(self.encoded_layers_[-1][batch_i])):
            # Holds 12 layers of hidden states for each token 
            hidden_layers = [] 
            # For each of the 12 layers...
            for layer_i in range(len(self.encoded_layers_)):
            # Lookup the vector for `token_i` in `layer_i`
                vec = self.encoded_layers_[layer_i][batch_i][token_i]
                hidden_layers.append(vec)
            token_embeddings.append(hidden_layers)
        self.token_embeddings_ = token_embeddings
    
    def feed_forward(self, sentence):
        self._tokenize_text(sentence)
        self._evaluate()
        self._generate_token_embeddings()
    
    def create_word_embedding_(self, how="cat_last_4"):
        if how == "cat_last_4":
            return [torch.cat((layer[-1], layer[-2], layer[-3], layer[-4]), 0) for layer in self.token_embeddings_]
        elif how == "sum_last_4":
            return [torch.sum(torch.stack(layer)[-4:], 0) for layer in self.token_embeddings_]
        else:
            print("Redefine `how` parameter")
        
    def create_sentence_embedding_(self, how="mean_last_layer"):
        if how == "mean_last_layer":
            return torch.mean(self.encoded_layers_[-1], 1).squeeze()
        elif how == "mean_cat_last_4_layers":
            return torch.mean(torch.cat((self.encoded_layers_[-1], self.encoded_layers_[-2], self.encoded_layers_[-3], self.encoded_layers_[-4]), 2), 1).squeeze()
            #return torch.mean(torch.stack(self.create_word_embedding_("cat_last_4")), 0)
        elif how == "mean_sum_last_4_layers": 
            return torch.mean(torch.sum(torch.stack(self.encoded_layers_[-4:]), 0), 1).squeeze()
        else:
            print("Redefine `how` parameter", how)
            
    def print_dimensions_(self):
        print ("Number of layers:", len(self.encoded_layers_))
        layer_i = 0
        print ("Number of batches:", len(self.encoded_layers_[layer_i]))
        batch_i = 0
        print ("Number of tokens:", len(self.encoded_layers_[layer_i][batch_i]))
        token_i = 0
        print ("Number of hidden units:", len(self.encoded_layers_[layer_i][batch_i][token_i]))
        
    def plot_embedding_hist(self, vec):
        sns.set_style("whitegrid")
        plt.figure(figsize=(12, 8))
        sns.distplot(vec)
        plt.show()

## Text Preprocessing

In [42]:
import os

import pandas as pd
import numpy as np

from tqdm import tqdm

In [259]:
import codecs
from pprint import pprint
NUMBER_OF_FILES_TO_OPEN = 4998
CORPUS = "onet"

In [260]:
corpusNews = list()
for i in range(0,NUMBER_OF_FILES_TO_OPEN):
    try:
        f = codecs.open("doc/"+ CORPUS +"/doc"+str(i+1), "r", encoding = 'utf-8')
        lines = ""
        for line in f:
            lines += line
        corpusNews.append(lines)
        f.close()
    except FileNotFoundError:
        with open("doc/"+ CORPUS +"/doc"+str(i+1), "w", encoding = 'utf-8') as missing:
            pass
        corpusNews.append("")
        
pprint(corpusNews[0])
print(len(corpusNews))

('Według Loose tematem spotkania <Entity name="Jarosław Gowin" type="person" '
 'category="politycy">Gowina</Entity> z kierownictwem klubu PiS będą '
 'propozycje ustawowe ministerstwa sprawiedliwości dotyczące m.in. deregulacji '
 'zawodów oraz planowanych zmian w Kodeksie postępowania karnego. Za trzy '
 'tygodnie, 10 i 11 grudnia, ma odbyć się wysłuchanie publiczne w sprawie '
 'projektu częściowo lub całkowicie deregulującego dostęp do 50 zawodów - '
 'zdecydowała we wtorek na swym pierwszym merytorycznym posiedzeniu sejmowa '
 'komisja do spraw deregulacji. Wniosek w sprawie wysłuchania - zgłoszony '
 'przez przewodniczącego komisji <Entity name="Adam Szejnfeld" type="person" '
 'category="politycy">Adama Szejnfelda</Entity> (PO) - poparli wszyscy '
 'członkowie komisji. Podczas wysłuchania głos będą mogli zabrać '
 'przedstawiciele wszystkich zainteresowanych samorządów zawodowych i '
 'organizacji społecznych. Ich reprezentanci będą też uczestniczyć w '
 'posiedzeniach komisji d

Below are functions from Piter, that I did not changed (but use it differently):

In [12]:
def find_annotations(document : str):
    '''
    Searches for all occurances of '<' and '>' in the document.
    Returns lists of indexes of occurances opening for '<' and closing for '>'
    '''
    i = 0
    opening = list()
    closing = list()
    while i != -1:
        i = document.find('<', i)
        opening.append(i)
        if i == -1:
            closing.append(-1)
            break
        i = document.find('>', i)
        closing.append(i)
    closing = [cl + 1 for cl in closing]
    closing[-1] = -1
    return(opening, closing)

def get_annotation_values(text : str): # jest typ nie type, bo type jest zarezerwowana nazwa, nie jestem uposledzony
    '''
    Returns a dict consisting annotation values {'name', 'typ', 'category'} for first occuring annotation
    in the text.
    '''
    name_start = text.find('name=') + len('name=\"')
    name_end = text.find('\"', name_start)
    typ_start = text.find('type=', name_end) + len('type=\"')
    typ_end = text.find('\"', typ_start)
    category_start = text.find('category', typ_end) + len('category=\"')
    category_end = text.find('\"', category_start)
    return({'name' : text[name_start:name_end], 'typ' : text[typ_start:typ_end],
               'category' : text[category_start:category_end]})

def exclude_vectors_nsize(text, nsize = 3):
    '''
    Parameters:
    text - document string
    nsize - size of window around person word 
    Returns a list of lists build as follows [k_words before person, person, k_words after person], person name]
    '''
    global people_dict
    opn, cls = find_annotations(text)
    ind = 0
    l = list()
    for i in range(0, len(opn) - 1, 2):
        left_sentence = last_n(text[ind:opn[i]].split(' '), nsize)
        left_sentence = repair_sentence(left_sentence, nsize, left = True)
        right_sentence = first_n(text[cls[i+1]:text.find('<', cls[i+1])].split(' ') , nsize)
        right_sentence = repair_sentence(right_sentence, nsize, left = False)
        annotation = get_annotation_values(text[ind:-1])
        l.append([flat_list([left_sentence, right_sentence]), annotation["name"]])
        try:
            people_dict[annotation["category"]].add(annotation["name"])
        except KeyError:
            people_dict[annotation["category"]] = {annotation["name"]}
        ind = cls[i+1]
    return l

def exclude_vectors_for_person(list_of_vectors, person):
    '''
    list_of_vectors - return value from exclude_sentence_vectors()
    person - name of person from people_dict
    '''
    l = list()
    for el in list_of_vectors:
        if el[1] == person:
            l.append(el[0])
    return l

More functions from Piter:

In [13]:
def reverse(text : str):
    '''
    Returns reversed text.
    '''
    return(text[::-1])

def last_n(_list, n):
    '''
    Returns last n elements of list. 
    Returns full list if n is greater than list length or empty string if list is empty
    '''
    if not _list:
        return('')
    if _list[len(_list) - 1] == '':
        _list.pop()
    return(_list[-n:])

def first_n(_list, n):
    '''
    Returns first n elements of list. 
    Returns full list if n is greater than list length or empty string if list is empty
    '''
    if not _list:
        return('')
    if _list[0] == '':
        _list.pop(0)
    return(_list[:n])


In [14]:
words_with_dot = ['m.in.', 'inż.', 'prof.', 'tzn.', 'np.', 'cd.', 'al.', 'cnd.', 
                  'itp.', 'itd.', 'lek.', 'lic.', 'pl.', 'p.o.', 'św.', 'tj.', 
                  'tzw.', 'ul.', 'zob.', 'ul.']

punctuation = ['"', ',', '.', ':', '(', ')', '?', '!']

In [15]:
def split_the_word(word = '"Pas.chanacz:lolo)u(marek,pies!?"'):
    '''
    Splits the word with elements in punctuation list. Words in words_with_dot are excluded from splitting.
    Returns a list of splitted word. Splitting characters are included in the list.
    '''
    global words_with_dot
    global punctuation
    if word in words_with_dot:
        return([word])
    else:
        l = list()
        index_start = 0
        index_end = 0
        for i, char in enumerate(word):
            if char in punctuation:
                if index_start != index_end:
                    l.append(word[index_start:index_end])
                l.append(char)
                index_start = i + 1
                index_end = i + 1
            else:
                index_end = i + 1
        if index_start != index_end:
            l.append(word[index_start:index_end])
        return(l)
    
def flat_list(_list): # https://stackoverflow.com/questions/952914/how-to-make-a-flat-list-out-of-list-of-lists
    '''
    Create one list from list of lists.
    '''
    flat_list = []
    for sublist in _list:
        for item in sublist:
            flat_list.append(item)
    return(flat_list)
    
def repair_sentence(_list, nsize, left):
    '''
    Naprawia zdanie xD. Chodzi o to, zeby oddzielic znaki interpunkcyjne.
    '''
    global words_with_dot
    l = list()
    for el in _list:
        if el not in words_with_dot:
            l.append(split_the_word(el))
        else:
            l.append([el])
    if left:
        return(last_n(flat_list(l), nsize))
    else:
        return(first_n(flat_list(l), nsize))

In [16]:
people_dict = {}

def exclude_vectors_nsize(text, nsize = 3):
    '''
    Parameters:
    text - document string
    nsize - size of window around person word 
    Returns a list of lists build as follows [k_words before person, person, k_words after person], person name]
    '''
    global people_dict
    opn, cls = find_annotations(text)
    ind = 0
    l = list()
    for i in range(0, len(opn) - 1, 2):
        left_sentence = last_n(text[ind:opn[i]].split(' '), nsize)
        left_sentence = repair_sentence(left_sentence, nsize, left = True)
        right_sentence = first_n(text[cls[i+1]:text.find('<', cls[i+1])].split(' ') , nsize)
        right_sentence = repair_sentence(right_sentence, nsize, left = False)
        annotation = get_annotation_values(text[ind:-1])
        l.append([flat_list([left_sentence, right_sentence]), annotation["name"]])
        try:
            people_dict[annotation["category"]].add(annotation["name"])
        except KeyError:
            people_dict[annotation["category"]] = {annotation["name"]}
        ind = cls[i+1]
    return l, people_dict

And our function for sentences:

In [17]:
def exclude_sentence_vectors(text):
    people_dict = {}
    sentences = text.split(".")
    contexts = []
    for sentence in sentences:
        opn, cls = find_annotations(sentence)
        if opn[0] != -1:
            #print(opn, cls)
            annotation = get_annotation_values(sentence)
            sentence = sentence[0:opn[0]] + sentence[cls[-2]:]
            contexts.append([sentence, annotation["name"]])
            try:
                people_dict[annotation["category"]].add(annotation["name"])
            except KeyError:
                people_dict[annotation["category"]] = {annotation["name"]}
    return contexts, people_dict


## vectors, people_dict = exclude_sentence_vectors(corpusNews[0])
print(vectors)
print(people_dict)

In [18]:
vectors, people_dict = exclude_vectors_nsize(corpusNews[0])
print(vectors) 
print(people_dict)
print(exclude_vectors_for_person(vectors, "Tomasz Sekielski"))

[[['Tomasza', 'oburzyła', 'wypowiedź', 'prof.'], 'Tomasz Sekielski'], [['prawa', '-', 'mówił', 'w', 'TOK', 'FM'], 'Tomasz Sekielski'], [['Wyborczej', '"', 'prof.', 'komentował', 'informacje', 'o'], 'Marian Filar'], [['zaskoczyły', 'i', 'oburzyły', '.', 'Dziennikarz', 'TVP'], 'Tomasz Sekielski'], [['"', '.', 'Zdaniem', 'prawnik', 'z', 'UMK'], 'Tomasz Sekielski'], [['morderca-pedofil', '-', 'powiedział'], 'Tomasz Sekielski']]
{'dziennikarze': {'Tomasz Sekielski'}, 'muzycy': {'Marian Filar'}}
[['Tomasza', 'oburzyła', 'wypowiedź', 'prof.'], ['prawa', '-', 'mówił', 'w', 'TOK', 'FM'], ['zaskoczyły', 'i', 'oburzyły', '.', 'Dziennikarz', 'TVP'], ['"', '.', 'Zdaniem', 'prawnik', 'z', 'UMK'], ['morderca-pedofil', '-', 'powiedział']]


In [19]:
vectors = exclude_vectors_nsize(corpusNews[0], 4)
pprint(vectors)

([[['Tomasza', 'oburzyła', 'wypowiedź', 'prof.', 'Mariana'],
   'Tomasz Sekielski'],
  [['łamać', 'prawa', '-', 'mówił', 'w', 'TOK', 'FM', '.'], 'Tomasz Sekielski'],
  [['Gazety',
    'Wyborczej',
    '"',
    'prof.',
    'komentował',
    'informacje',
    'o',
    'znalezieniu'],
   'Marian Filar'],
  [['prawnika',
    'zaskoczyły',
    'i',
    'oburzyły',
    '.',
    'Dziennikarz',
    'TVP',
    'przyznał'],
   'Tomasz Sekielski'],
  [['FM', '"', '.', 'Zdaniem', 'prawnik', 'z', 'UMK', 'w'], 'Tomasz Sekielski'],
  [['tylko', 'morderca-pedofil', '-', 'powiedział'], 'Tomasz Sekielski']],
 {'dziennikarze': {'Tomasz Sekielski'}, 'muzycy': {'Marian Filar'}})


In [20]:
people_dict

{'dziennikarze': {'Tomasz Sekielski'}, 'muzycy': {'Marian Filar'}}

## Generating tsv files

In [98]:
NAME = "Name"
EMBEDDING = "embedding"
PROFESSION = "Profession"
TYPE = "mean_cat_last_4_layers"


In [371]:
def generate_embeddings_words(news_file):
    embeddings = []
    metadata = []
    vectors, people_dict = exclude_vectors_nsize(news_file, 4)
    for profession in people_dict:
        names = people_dict[profession]
        for name in names:
            person_contexts = exclude_vectors_for_person(vectors, name)
            for context in person_contexts:
                concatenated = " ".join(c for c in context)
                bert.feed_forward(concatenated)
                sent_embedding = bert.create_sentence_embedding_(how = TYPE)
                embeddings.append(sent_embedding)
                metadata.append({NAME: name, PROFESSION: profession})
    return embeddings, metadata
        

In [370]:
def generate_embeddings_sentence(news_file):
    embeddings = []
    metadata = []
    vectors, people_dict = exclude_sentence_vectors(news_file)
    for profession in people_dict:
        names = people_dict[profession]
        for name in names:
            person_contexts = exclude_vectors_for_person(vectors, name)
            for context in person_contexts:
                bert.feed_forward(context)
                sent_embedding = bert.create_sentence_embedding_(how = TYPE)
                embeddings.append(sent_embedding)
                metadata.append({NAME: name, PROFESSION: profession})
    return  embeddings, metadata
        

In [396]:
def create_vextors_meta_df(number_of_layers=4):
    vectors_meta_df = pd.DataFrame(columns=range(number_of_layers*768+3)).rename(columns=
        {
        number_of_layers*768: "Name",
        number_of_layers*768+1: "Profession",
        number_of_layers*768+2: "Document"
        })
    return vectors_meta_df

def generate_concat_vectors_meta_df(number_of_files):
    vectors_meta_df = create_vextors_meta_df()
    for i in tqdm(range(0, number_of_files)):
        #pprint(corpusNews[i])
        embeddings, metadata = generate_embeddings_words(corpusNews[i])
        frames = []
        for e in embeddings:
            values = [t.item() for t in e]
            frames.append(pd.DataFrame(values).T)
        try:
            fr = pd.concat(frames)
            joined = fr.join(pd.DataFrame(metadata))
            joined["Document"] = "doc"+ str(i+1)
            vectors_meta_df = vectors_meta_df.append(joined)
        except ValueError:
            continue
            
    return vectors_meta_df


In [399]:
df = generate_concat_vectors_meta_df(NUMBER_OF_FILES_TO_OPEN)
df.to_csv("tsv_files/" + TYPE +"/" + CORPUS +"/results_words1.tsv", sep="\t")


  8%|▊         | 417/4998 [00:50<14:42,  5.19it/s]


 17%|█▋        | 863/4998 [01:34<11:08,  6.18it/s]


 24%|██▎       | 1186/4998 [02:27<09:53,  6.42it/s]


 31%|███▏      | 1565/4998 [03:07<02:53, 19.75it/s]


 39%|███▉      | 1937/4998 [03:57<06:33,  7.78it/s]


...

 71%|███████   | 3556/4998 [07:01<03:39,  6.57it/s]


 79%|███████▉  | 3951/4998 [07:48<02:11,  7.95it/s]


 88%|████████▊ | 4387/4998 [08:34<02:26,  4.18it/s]


 93%|█████████▎| 4673/4998 [09:22<00:41,  7.78it/s]


100%|██████████| 4998/4998 [10:11<00:00,  8.17it/s]


In [398]:
df= pd.read_csv("tsv_files/" + TYPE +"/" + CORPUS +"/results_words1.tsv", sep="\t")
df.groupby(["Document", "Profession", "Name"]).mean().reset_index()

,Document,Profession,Name,Unnamed: 0,0,1,2,3,4,5,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,doc1,politycy,Jarosław Gowin,0,-0.077040,-0.280966,0.888300,0.159213,0.245626,0.056108,...,0.080211,-0.552011,-0.368075,-0.269050,0.388620,-0.389159,-0.556412,-0.038339,0.084515,0.260660
1,doc10,politycy,Maciej Rataj,0,-0.020054,-0.284341,0.463016,0.084573,0.338564,0.162313,...,0.159499,-0.388542,-0.749628,-0.477634,0.385759,-0.376551,-0.421735,0.052833,0.489614,0.217435
2,doc101,politycy,Irena Lipowicz,0,-0.105039,-0.296177,0.631779,0.111769,0.215479,0.265545,...,0.289515,-0.605164,-0.632973,-0.576649,0.332326,-0.460350,-0.496762,0.123326,0.203858,-0.101886
3,doc102,politycy,Donald Tusk,0,0.140507,-0.169071,0.364256,0.080722,0.384109,0.361359,...,0.289154,-0.500774,-0.549308,-0.341060,0.213128,-0.360049,-0.388345,0.332273,0.487607,0.087700
4,doc103,politycy,Joachim Brudziński,0,-0.025080,-0.288531,0.529188,0.138812,0.329439,0.141213,...,0.123808,-0.153015,-0.557275,-0.366970,0.220043,-0.469725,-0.580212,0.030807,0.273209,0.257715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,doc94,politycy,Waldemar Pawlak,0,-0.007887,-0.293649,0.437457,0.156701,0.340602,0.091654,...,0.087135,-0.139974,-0.659284,-0.553864,0.254653,-0.489999,-0.823888,-0.305623,-0.055890,0.362783
164,doc95,politycy,Włodzimierz Cimoszewicz,0,-0.067523,-0.330465,0.511320,0.000298,0.286162,0.271232,...,0.118256,-0.672239,-0.390406,-0.665017,0.473673,-0.410814,-0.529500,0.054179,0.165175,-0.106471
165,doc96,dziennikarze,Jerzy Giedroyc,0,0.036858,-0.314028,0.372681,0.051369,0.335927,0.161099,...,0.201822,-0.215329,-0.532317,-0.441958,0.221541,-0.465711,-0.434263,0.139180,0.407487,0.157164
166,doc98,politycy,Stanisław Żelichowski,0,0.005060,-0.243434,0.417410,0.121783,0.260104,0.215375,...,0.174754,-0.513138,-0.605559,-0.462636,0.372224,-0.375303,-0.466680,0.130805,0.329049,0.057914


## Example usage

1. Create API object

In [ ]:
bert = BertBaseMultilingualEmbeddingApi()

2. Create sentence and pass to feed_forward method

In [ ]:
sentence = "Oszczędnością i pracą ludzie się bogacą."
bert.feed_forward(sentence)

3. Generate word/sentence embedding, specify `how`

In [ ]:
words_embeddings = bert.create_word_embedding_(how = "sum_last_4")

In [ ]:
sent_embedding = bert.create_sentence_embedding_()

4. Visualize tensor

In [ ]:
print(sent_embedding)

In [ ]:
bert.plot_embedding_hist(sent_embedding)

## Visualize embeddings in tensorboard

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

In [ ]:
LOG_DIR = 'minimalsample'
NAME_TO_VISUALISE_VARIABLE = "example_embeddings"
path_for_mnist_metadata =  'metadata.tsv'

In [ ]:
token_embeddings_np = torch.stack(concatenated_lats_4_layers).numpy()

In [ ]:
embedding_var = tf.Variable(token_embeddings_np, name=NAME_TO_VISUALISE_VARIABLE)
summary_writer = tf.summary.FileWriter(LOG_DIR)

In [ ]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

# Specify where you find the metadata
embedding.metadata_path = path_for_mnist_metadata #'metadata.tsv'

# Say that you want to visualise the embeddings
projector.visualize_embeddings(summary_writer, config)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.save(sess, os.path.join(LOG_DIR, "model.ckpt"), 1)

In [ ]:
with open(path_for_mnist_metadata,'w') as f:
    f.write("Index\tLabel\n")
    for index,label in enumerate(tokenized_text):
        f.write("%d\t %s\n" % (index,label.encode('utf-8')))